pip install -r requirements.txt (가상환경에 이미 설치됨)

주피터노트북 커널 연결(python -m ipykernel install --user --name myenv --display-name corpus)

한글 폰트(주피터랩 자체를 다시 켜야지 폰트 적용되는 것 같음.)

apt-get update

apt-get install -y fonts-nanum

# bareun download(bareunpy-1.6.6)

In [ ]:
# 바른 형태소 분석기 리눅스 설치
!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb

In [ ]:
# 설치
!uname -a
!dpkg -i bareun-linux.deb

In [ ]:
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg koba-VAD6MMA-BFSU7XI-SYQYWYY-Y4MXHVA

In [ ]:
# 실행 정보
!ps -ef | grep bareun

# library

In [ ]:
import bareunpy as brn
import google.protobuf.text_format as tf
import sys
import tqdm
import pandas as pd
import pyarrow
import fastparquet

# bareun test

In [ ]:
# 아래에 "https://bareun.ai/"에서 이메일 인증 후 발급받은 API KEY("koba-...")를 입력해주세요. "로그인-내정보 확인"
API_KEY = "koba-VAD6MMA-BFSU7XI-SYQYWYY-Y4MXHVA" # <- 본인의 API KEY로 교체
t = brn.Tagger(API_KEY, "localhost", 5656) # tagger는 변수 t에 할당
res = t.tags(["안녕하세요. 정말 좋은 날씨네요."]) # t.tags()에 리스트로 한 문장을 입력
m = res.msg() # 분석 결과를 객체로 저장하여 변수 m에 할당

tf.PrintMessage(m, out=sys.stdout, as_utf8=True) # 분석 결과 객체 m을 가독성 좋게 표시.

# 데이터 확인

In [ ]:
def tag_sentences_with_bareun(df, api_key, host='localhost', port=5656):
    # Bareun Tagger 초기화
    t = brn.Tagger(api_key, host, port)

    # 총 문장 수 출력
    print(f"Total number of sentences to tag: {len(df)}")
    
    # 각 문장 태깅 함수
    def tag_sentence(sentence):
      try:
          res = t.tags([sentence])
          return list(res.pos())
      except Exception as e:
          print(f"Error tagging sentence: {sentence}")
          print(f"Error: {e}")
          return []

    tagged_sentences = []
    for sentence in tqdm.tqdm(df['sentence'], desc="Tagging Sentences", mininterval=10):
        tagged_sentences.append(tag_sentence(sentence))
    
    # 새로운 'tagged_sentence' 열 추가
    df['tagged_sentence'] = tagged_sentences
    
    return df

In [19]:
df = pd.read_parquet('/workspace/NIKL_NEWSPAPER_PARQUET/NIKL_NEWSPAPER_37.parquet')

In [25]:
# NoneType에 해당하는 행은 제외
df = df[df['sentence'].notnull()]

In [33]:
# 하나의 문자열로 이루어져 있지만 형태소 분석이 어렵고, 유의미한 문장으로 보이지 않으므로 제거하기.
df.iloc[17649]['sentence']

'분류        수사 대상자        혐의        처리 결과                      문건 관련(피고소인)        유아무개씨 (ㅎ기획사 대표)        출판물에 의한 명예훼손        불구속 입건                      <한국방송> 기자 2명        사자 명예훼손        무혐의                                     성상납의혹        김아무개씨(장자연씨 소속사 대표)        강요·폭행 등        기소중지                                     유력인사(피고소인)        <조선일보> 고위임원        강요죄 공범        무혐의                      금융인 ㄱ씨        강요죄 공범        참고인 중지                      기업인 ㄴ씨        강요죄 공범        참고인 중지                                     장자연 문건 등장 인물        언론인 ㄷ씨 (<조선일보> 고위임원 아들)        강요죄 공범        내사중지                      드라마 감독 3명        강요죄 공범        내사중지                      드라마 감독 1명        강요죄 공범        혐의 없음                                     수사로밝혀낸새인물(8명)        드라마 감독 ㄹ씨        강요죄 공범, 배임수재        불구속 입건                      금융인 ㅁ씨        강제추행        불구속 입건                      금융인 ㅂ씨        강요죄 공범        참고인 중지                      연예기획사 대표 ㅅ씨, 드라마 감독 ㅇ씨        강요죄 공범        참고인 중지        

In [34]:
df = df.drop(df.iloc[17649].name)

In [35]:
len(df)

999998

In [36]:
tagged_df = tag_sentences_with_bareun(df, API_KEY)

Total number of sentences to tag: 999998


Tagging Sentences: 100%|██████████| 999998/999998 [5:09:16<00:00, 53.89it/s]  


In [37]:
# parquet 파일로 저장
tagged_df.to_parquet('/workspace/tagged_NIKL_NEWSPAPER_PARQUET/tagged_NIKL_NEWSPAPER_37.parquet', index=False)